# RCE - geographical metadata exploration 

## Explore

In [2]:
import pandas as pd
import zipfile

In [3]:
### Inspect zip file contents 

# Path to zip file
zip_file_path = '../data/metadata-2.zip'
#zip_file_path = '../data/metadata-1.zip'

# Open the zip file and list its contents
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_contents = zip_ref.namelist()  # List all files inside the zip
    print("Files in the ZIP archive:")
    for file in zip_contents:
        print(file)



Files in the ZIP archive:
metadata-16-01-2025/
metadata-16-01-2025/getmetadata.sh
metadata-16-01-2025/metadata-from-prod-server-0.csv
metadata-16-01-2025/metadata-from-prod-server-1.csv
metadata-16-01-2025/metadata-from-prod-server-10.csv
metadata-16-01-2025/metadata-from-prod-server-11.csv
metadata-16-01-2025/metadata-from-prod-server-12.csv
metadata-16-01-2025/metadata-from-prod-server-13.csv
metadata-16-01-2025/metadata-from-prod-server-14.csv
metadata-16-01-2025/metadata-from-prod-server-15.csv
metadata-16-01-2025/metadata-from-prod-server-16.csv
metadata-16-01-2025/metadata-from-prod-server-17.csv
metadata-16-01-2025/metadata-from-prod-server-18.csv
metadata-16-01-2025/metadata-from-prod-server-19.csv
metadata-16-01-2025/metadata-from-prod-server-2.csv
metadata-16-01-2025/metadata-from-prod-server-20.csv
metadata-16-01-2025/metadata-from-prod-server-21.csv
metadata-16-01-2025/metadata-from-prod-server-22.csv
metadata-16-01-2025/metadata-from-prod-server-23.csv
metadata-16-01-2025/

In [4]:
# Paths to the zip files
zip_file_1 = '../data/metadata-1.zip'
zip_file_2 = '../data/metadata-2.zip'

In [5]:
# List to store DataFrames from both ZIPs
dfs = []

# Read CSV files from a ZIP file and add them to the list
def read_csv_from_zip(zip_file):
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        # Get a list of all the files in the zip
        zip_contents = zip_ref.namelist()
        
        # Filter out only the CSV files
        csv_files = [f for f in zip_contents if f.endswith('.csv')]
        
        # Read each CSV file and append to the DataFrame list
        for csv_file in csv_files:
            with zip_ref.open(csv_file) as file:
                df = pd.read_csv(file)
                dfs.append(df)  # Append the DataFrame

# Read CSVs from both zip files
read_csv_from_zip(zip_file_1)
read_csv_from_zip(zip_file_2)

# Concatenate DataFrames into a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

In [6]:
combined_df.columns

Index(['dsPersistentId', 'publicationStatus', 'title', 'dsDescriptionValue',
       'dccd-projectType', 'dccd-category', 'dccd-taxon', 'dccd-objectType',
       'dccd-elementType', 'dccd-latitude', 'dccd-longitude',
       'dccd-periodStart', 'dccd-periodEnd', 'metadata_type_ss',
       'dansSpatialPointX', 'dansSpatialPointY', 'dansSpatialPointScheme',
       'dansSpatialBoxNorth', 'dansSpatialBoxEast', 'dansSpatialBoxSouth',
       'dansSpatialBoxWest', 'dansSpatialBoxScheme',
       'dansSpatialCoverageControlleddansSpatialCoverageText'],
      dtype='object')

In [7]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169705 entries, 0 to 169704
Data columns (total 23 columns):
 #   Column                                                Non-Null Count   Dtype  
---  ------                                                --------------   -----  
 0   dsPersistentId                                        169705 non-null  object 
 1   publicationStatus                                     169705 non-null  object 
 2   title                                                 169705 non-null  object 
 3   dsDescriptionValue                                    169389 non-null  object 
 4   dccd-projectType                                      5337 non-null    object 
 5   dccd-category                                         5337 non-null    object 
 6   dccd-taxon                                            5033 non-null    object 
 7   dccd-objectType                                       5117 non-null    object 
 8   dccd-elementType                            

In [8]:
# Inspect values of publicationStatus
pubstatus = combined_df.publicationStatus.value_counts()
print(pubstatus)

publicationStatus
Published                      166283
Unpublished,Draft                2796
Unpublished,Draft,In Review       247
Draft                             199
Deaccessioned                     174
Draft,In Review                     6
Name: count, dtype: int64


In [9]:
# Count missing values 
nan_counts = combined_df.isna().sum()


In [10]:
# Select archeological datasets
arch_df = combined_df[combined_df.dansSpatialPointX.notna() | combined_df.dansSpatialBoxNorth.notna()]


In [11]:
arch_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60313 entries, 10416 to 169704
Data columns (total 23 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   dsPersistentId                                        60313 non-null  object 
 1   publicationStatus                                     60313 non-null  object 
 2   title                                                 60313 non-null  object 
 3   dsDescriptionValue                                    60313 non-null  object 
 4   dccd-projectType                                      0 non-null      object 
 5   dccd-category                                         0 non-null      object 
 6   dccd-taxon                                            0 non-null      object 
 7   dccd-objectType                                       0 non-null      object 
 8   dccd-elementType                                      0 

In [12]:
# Select only published datasets
df_pub = arch_df[arch_df.publicationStatus == 'Published']

In [13]:
df_pub.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59837 entries, 10416 to 169704
Data columns (total 23 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   dsPersistentId                                        59837 non-null  object 
 1   publicationStatus                                     59837 non-null  object 
 2   title                                                 59837 non-null  object 
 3   dsDescriptionValue                                    59837 non-null  object 
 4   dccd-projectType                                      0 non-null      object 
 5   dccd-category                                         0 non-null      object 
 6   dccd-taxon                                            0 non-null      object 
 7   dccd-objectType                                       0 non-null      object 
 8   dccd-elementType                                      0 

In [14]:
# Make a list of DOIs
dois = df_pub.dsPersistentId.tolist()

In [15]:
dois[:10]

['doi:10.17026/dans-zrj-unr7',
 'doi:10.17026/dans-299-9dpm',
 'doi:10.17026/dans-zqy-ymw8',
 'doi:10.17026/dans-z8d-9c6h',
 'doi:10.17026/dans-x9v-j3qu',
 'doi:10.17026/dans-2xv-vrz6',
 'doi:10.17026/dans-zfx-fuem',
 'doi:10.17026/dans-z4f-4hp3',
 'doi:10.17026/dans-24x-uv4g',
 'doi:10.17026/dans-zm6-226h']

## Get OAI-ORE metadata

In [16]:
import requests
import json
import pprint

In [17]:
# URL of the JSON file
url = "https://dataverse.nl/api/datasets/export?exporter=OAI_ORE&persistentId=doi%3A10.34894/1TQV3K"

try:
    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    response.raise_for_status()

    # Parse the JSON data
    data = response.json()

    # Optionally, save it to a file
    with open('../jsons/test.json', 'w') as json_file:
        json.dump(data, json_file, indent=4)

    print("JSON data has been saved to 'test.json'.")
    pprint.pprint(data) 

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

JSON data has been saved to 'test.json'.
{'@context': {'author': 'http://purl.org/dc/terms/creator',
              'authorIdentifier': 'http://purl.org/spar/datacite/AgentIdentifier',
              'authorIdentifierScheme': 'http://purl.org/spar/datacite/AgentIdentifierScheme',
              'citation': 'https://dataverse.org/schema/citation/',
              'content': '@value',
              'dansDataVaultMetadata': 'https://dataverse.nl/schema/dansDataVaultMetadata#',
              'dateOfDeposit': 'http://purl.org/dc/terms/dateSubmitted',
              'dccd': 'https://dataverse.nl/schema/dccd#',
              'dcterms': 'http://purl.org/dc/terms/',
              'dvcore': 'https://dataverse.org/schema/core#',
              'lang': '@language',
              'ore': 'http://www.openarchives.org/ore/terms/',
              'schema': 'http://schema.org/',
              'scheme': 'http://www.w3.org/2004/02/skos/core#inScheme',
              'subject': 'http://purl.org/dc/terms/subject',
